In [1]:
import pandas as pd 
import numpy as np 
import json 
from ast import literal_eval

def flatten(in_csv, out_csv, nrows=None):
    df = pd.read_csv(in_csv, dtype=np.object, nrows=nrows)

    # customDimensions
    def parse_customDimensions(raw_str):
        lst = literal_eval(raw_str)
        if isinstance(lst, list) and lst:
            return pd.Series(lst[0])
        else:
            return pd.Series({})
    
    customDimensions_df = df.customDimensions.apply(parse_customDimensions)
    customDimensions_df.columns = [f'customDimensions_{x}' for x in customDimensions_df.columns]
    df = pd.concat([df, customDimensions_df], axis=1)
    df.drop('customDimensions', axis=1, inplace=True)

    # hits
    def parse_hits(raw_str):
        lst = literal_eval(raw_str)
        if isinstance(lst, list) and lst:
            return pd.Series(lst[0])
        else:
            return pd.Series({})
    
    hits_df = df.hits.apply(parse_hits)
    hits_df.columns = [f'hits_{x}' for x in hits_df.columns]
    df = pd.concat([df, hits_df], axis=1)
    df.drop('hits', axis=1, inplace=True)

    # 'hits_page', 'hits_transaction', 'hits_item', 'hits_appInfo',
    # 'hits_exceptionInfo', 'hits_eCommerceAction', 'hits_social', 'hits_contentGroup', 'hits_promotionActionInfo'
    dict_cols = ['hits_page', 'hits_transaction', 'hits_item', 'hits_appInfo', 
        'hits_exceptionInfo', 'hits_eCommerceAction', 'hits_social', 'hits_contentGroup', 'hits_promotionActionInfo']
    for col in dict_cols:
        parsed_df = hits_df[col].apply(pd.Series)
        parsed_df.columns = [f'{col}_{x}' for x in parsed_df.columns]
        df = pd.concat([df, parsed_df], axis=1)
        df.drop(col, axis=1, inplace=True)
    
    # 'hits_experiment', 'hits_customVariables', 'hits_customMetrics', 'hits_publisher_infos', 'hits_customDimensions' are empty
    df.drop(['hits_experiment', 'hits_customVariables', 'hits_customMetrics', 'hits_publisher_infos', 'hits_customDimensions'], axis=1, inplace=True)

    # 'hits_product', 'hits_promotion'
    def parse_list(x):
        if isinstance(x, list) and x:
            return pd.Series(x[0])
        else:
            return pd.Series({})
    
    for col in ['hits_product', 'hits_promotion']:
        parsed_df = hits_df[col].apply(parse_list)
        parsed_df.columns = [f'{col}_{x}' for x in parsed_df.columns]
        df = pd.concat([df, parsed_df], axis=1)
        df.drop(col, axis=1, inplace=True)

    df.to_csv(out_csv, index=False)

    return df.shape

In [2]:
%%time
train_shape = flatten(in_csv='train_v2.csv', out_csv='flattened_train_v2.csv')

/anaconda/envs/py36/lib/python3.6/site-packages/pandas/core/indexes/api.py:69: RuntimeWarning: '<' not supported between instances of 'int' and 'str', sort order is undefined for incomparable objects
  index = _union_indexes(indexes, sort=sort)
/anaconda/envs/py36/lib/python3.6/site-packages/pandas/core/indexes/api.py:107: RuntimeWarning: '<' not supported between instances of 'int' and 'str', sort order is undefined for incomparable objects
  result = result.union(other)
/anaconda/envs/py36/lib/python3.6/site-packages/pandas/core/indexes/api.py:107: RuntimeWarning: '<' not supported between instances of 'str' and 'int', sort order is undefined for incomparable objects
  result = result.union(other)
/anaconda/envs/py36/lib/python3.6/site-packages/pandas/core/indexes/api.py:69: RuntimeWarning: '<' not supported between instances of 'str' and 'int', sort order is undefined for incomparable objects
  index = _union_indexes(indexes, sort=sort)


CPU times: user 6h 11min 24s, sys: 26min 50s, total: 6h 38min 15s
Wall time: 6h 38min 51s


In [32]:
%%time
test_shape = flatten(in_csv='test_v2.csv', out_csv='flattened_test_v2.csv')

/anaconda/envs/py36/lib/python3.6/site-packages/pandas/core/indexes/api.py:69: RuntimeWarning: '<' not supported between instances of 'int' and 'str', sort order is undefined for incomparable objects
  index = _union_indexes(indexes, sort=sort)
/anaconda/envs/py36/lib/python3.6/site-packages/pandas/core/indexes/api.py:107: RuntimeWarning: '<' not supported between instances of 'int' and 'str', sort order is undefined for incomparable objects
  result = result.union(other)
/anaconda/envs/py36/lib/python3.6/site-packages/pandas/core/indexes/api.py:107: RuntimeWarning: '<' not supported between instances of 'str' and 'int', sort order is undefined for incomparable objects
  result = result.union(other)
/anaconda/envs/py36/lib/python3.6/site-packages/pandas/core/indexes/api.py:69: RuntimeWarning: '<' not supported between instances of 'str' and 'int', sort order is undefined for incomparable objects
  index = _union_indexes(indexes, sort=sort)


CPU times: user 1h 30min 56s, sys: 3min 57s, total: 1h 34min 53s
Wall time: 1h 35min 35s
